In [ ]:
!pip install yt-dlp
!pip install openai-whisper
!pip install ffmpeg-python
!pip install langchain
!pip install langchain-community
!pip install langchain-huggingface
!pip install langchain-google-genai
!pip install sentence-transformers
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 22.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.3 MB/s eta 0:00:00


In [ ]:
# here it downloads the whole video and then extracts the audio from it.

import os
import yt_dlp
import whisper

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

# === API Keys ===
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "hf_key"
os.environ["GOOGLE_API_KEY"] = "gemini_api_key"

# === Step 1: Download audio from YouTube ===
video_url = "https://www.youtube.com/watch?v=vXVRs_Sar6Y"
audio_filename = "temp_audio.mp3"

print("📥 Downloading audio...")
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'temp_audio.%(ext)s',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# === Step 2: Transcribe audio using Whisper ===
print("🧠 Transcribing with Whisper...")
model = whisper.load_model("base")
result = model.transcribe(audio_filename)
transcript = result["text"]

# === Step 3: Split Text ===
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([transcript])

# === Step 4: Embedding and Vector Store ===
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)

# === Step 5: Setup Retriever ===
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# === Step 6: Setup LLM & Prompt ===
llm = GoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.2)
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an expert summarizer. Based on the context below, answer the user's question accurately and concisely.

Context:
{context}

Question:
{question}

Answer:
"""
)

# === Step 7: Ask a Question ===
question = "Is the topic 'black hole' discussed in the video? If yes, then what is it?"
retrieved_docs = retriever.invoke(question)
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
final_prompt = prompt.format(context=context_text, question=question)

# === Step 8: Generate Answer ===
answer = llm.invoke(final_prompt)
print("\n📌 Answer:\n", getattr(answer, "content", answer))





📥 Downloading audio...
[youtube] Extracting URL: https://www.youtube.com/watch?v=vXVRs_Sar6Y
[youtube] vXVRs_Sar6Y: Downloading webpage
[youtube] vXVRs_Sar6Y: Downloading tv client config
[youtube] vXVRs_Sar6Y: Downloading player 612f74a3-main
[youtube] vXVRs_Sar6Y: Downloading tv player API JSON
[youtube] vXVRs_Sar6Y: Downloading ios player API JSON
[youtube] vXVRs_Sar6Y: Downloading m3u8 information
[info] vXVRs_Sar6Y: Downloading 1 format(s): 251
[download] Destination: temp_audio.webm
[download] 100% of   17.74MiB in 00:00:00 at 39.61MiB/s  
[ExtractAudio] Destination: temp_audio.mp3
Deleting original file temp_audio.webm (pass -k to keep)
🧠 Transcribing with Whisper...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 125MiB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


📌 Answer:
 Yes, the video extensively discusses black holes.  It explains that black holes are regions with immensely strong gravity, preventing even light from escaping.  The video uses the movie *Interstellar* as a springboard to explore theoretical concepts about black holes, including the possibility of five-dimensional spaces within them and the nature of singularities at their centers.  It also clarifies misconceptions about black holes destroying the universe, emphasizing that they are central to many galaxies and that maintaining distance is key to safety.
